#Final Project - Sequence Generation with Keras
####Matan Zubery - 316263938  Ohad Dali - 316452929

Our mission is to choose genomes, learn a model for each gender and generate new genomes based on the models.
The genomes is taken from [www.internationalgenome.org/data](http://www.internationalgenome.org/data).

The sequence : **genome** ,The type of the sequence : **male or female**


##Data Collection
The data is stored in FASTQ format. FASTQ is a text-based format for storing a biological squence and its corresponding quality scores. Both the sequence letter and quality score are each encoded with a single ASCII character for brevity.

We will need to use the biopython library to parse the files and read it properly.

In [1]:
from Bio import SeqIO

First, let's examine this data and recognize some issues to handle.

In [26]:
records = SeqIO.parse("data\male\SRR101470_2.fastq","fastq")
first_record = records.next()
print first_record
print "seq len: " + str(len(first_record.seq))

ID: SRR101470.1
Name: SRR101470.1
Description: SRR101470.1 61C53AAXX100218:2:1:995:8481/2
Number of features: 0
Per letter annotation for: phred_quality
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN', SingleLetterAlphabet())
seq len: 76


We can see that each record holds a part of the whole genom.
So we need to group those records into whole genom (at least part of it).


In [27]:
genom = first_record.seq
index = 0
limit = 30000
for record in records:
    genom = genom + record.seq
    if index == limit:
        break
    index = index + 1
    


In [28]:
print len(genom)
for i in range(0,3000,150):
    print genom[i:i+150]

2280152
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

The DNA is built from 4 bases : A T C G . The N we see in our genom means that the sequencer was not able to determine if the base was A,T,C or G so this read is useless.

In [35]:
genom1 = str(genom)
genom1 = genom1.replace('N','')
print len(genom1)
for i in range(0,3000,150):
    print genom1[i:i+150]

2195417
CTTTCTCCAAAGCTGGCGCATAAGACCAACAAAGCACTGAATCGAACTGGTAGTTATTACATAGAGTATTATCGTCGAAGCCCTATTGCTTCAAAAGATCTACTAAACAGTAAAATCATTCCCCTATCAAAGATTTGTCCACTCCCCGGT
TCGATTAGTCCAGTGGGCGACAAATACTGGACGGGGGTACGATAAAATGTTTGGGGGTAGATAAAATAACCTAAAAAAACACCCTCATGGACAAGGCATACATACACCACAAACCTCAAGCTCTTATGAAAGACAAATTAGCTGGAGTCA
CACCCTCTATTTATCATATTTAACCATGCTGAACACCGCCTCGCCATAGCTTGATCTAAGAGGAGAAATCATGACGCACGACCTTCTCATGTCATTTTGCTTCTTTTGCTCATACAATGTTATTCTATCTATACCAACGACCACCGGCTA
GTCTTACTTGGGATGTTAGAGTCGAGGATCTACCCTGCTACCCCTCCATCAACAGGGACTATGGGAGGACAAAAAGAAGAAACACTGTTAATTAAGAATGGCAGACGCTCCGATCCTTGACATCACACGAGCGGGGCTTCATGATGAGCA
GCCTGACCCCACACTTGATTGTCACTACACTACCCGAACTGAGGTTTGTGCCACACATAAAGCGCGAAAATCATTGAAGATAATGGATTTTGAAGAGTCACAACACTAAGCGAAATCCTGCTTACCCCCATTCCATCCACAATGACATAT
CAGTTTGCTCACATCAGAAAAATTATAACACAGACACAAACCTACTCACCAAAACAGAGCATAACATAGATGAAAAATTAATCTATAGCTTATATGACCAGGCTGATTTAGGCTTGAAGAGGTCGCGGCAGAGGATACAATACTTAGATA
CAAGTCAAAAACATCACATTAGTTTTTAGATTCCTTCATTCATCATAACTTAAACCTCATATCCTTTAATGTTGGTGGGCGCAACA

##Building the network
####Setup

In [2]:
import numpy
import theano
import keras
from keras import backend as k
k.set_image_dim_ordering('th')
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

In [ ]:
model = Sequential()
